# Урок 7. Интерпретация прогнозов модели (SHAP): объясняем поведение модели на отдельных наблюдениях

In [53]:
# Подключаем библиотеки

import pandas as pd
import numpy as np
import functions as func
from sklearn.model_selection import train_test_split

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score, log_loss

## 1. Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных.
## 2. Подобрать необходимое распределение и тип связи, при необходимости ознакомиться с документацией H2O.
## 3. Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков или функции от них и т.д.).
## 4. Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке), проанализировать вероятные проблемы.
## 5. Предложить способы их решения и/или попробовать их решить, улучшив результат

In [27]:
df = pd.read_csv('freMPL-R.csv', low_memory = False)

In [28]:
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis = 1, inplace = True)
df.dropna(axis = 1, how = 'all', inplace = True)
df.drop_duplicates(inplace = True)
df.reset_index(drop = True, inplace = True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
Exposure             115155 non-null float64
LicAge               115155 non-null int64
RecordBeg            115155 non-null object
RecordEnd            59455 non-null object
Gender               115155 non-null object
MariStat             115155 non-null object
SocioCateg           115155 non-null object
VehUsage             115155 non-null object
DrivAge              115155 non-null int64
HasKmLimit           115155 non-null int64
BonusMalus           115155 non-null int64
ClaimAmount          115155 non-null float64
ClaimInd             115155 non-null int64
ClaimNbResp          115155 non-null float64
ClaimNbNonResp       115155 non-null float64
ClaimNbParking       115155 non-null float64
ClaimNbFireTheft     115155 non-null float64
ClaimNbWindscreen    115155 non-null float64
OutUseNb             115155 non-null float64
RiskArea             115155 non-null float64
dtypes

In [30]:
df.head()

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea
0,0.083,332,2004-01-01,2004-02-01,Male,Other,CSP50,Professional,46,0,50,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
1,0.916,333,2004-02-01,NaN,Male,Other,CSP50,Professional,46,0,50,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
2,0.550,173,2004-05-15,2004-12-03,Male,Other,CSP50,Private+trip to office,32,0,68,0.0,0,0.0,2.0,0.0,0.0,0.0,0.0,7.0
3,0.089,364,2004-11-29,NaN,Female,Other,CSP55,Private+trip to office,52,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
4,0.233,426,2004-02-07,2004-05-01,Male,Other,CSP60,Private,57,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,7.0


In [31]:
# Т.к. в некоторых наблюдений ранее была замечена отрицательная величина, то для таких полюсов переменная "ClaimInd" принимает только значение 0.
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [32]:
#Заменим все отрицательные значения "ClaimAmount" нулями.
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

In [33]:
df['Gender'], GenderRef = func.series_factorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [34]:
df['MariStat'], MariStatRef = func.series_factorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


In [35]:
list(df['VehUsage'].unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [36]:
VU_dummies = pd.get_dummies(df['VehUsage'], prefix = 'VehUsg', drop_first = False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


In [37]:
df['SocioCateg'] = df['SocioCateg'].str.slice(0,4)
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis = 1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [38]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

In [39]:
df = df.select_dtypes(exclude=['object'])

In [40]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df['LicAgeSq'] = df.LicAge.apply(lambda x: x**2)
df['BonusMalusSq'] = df.BonusMalus.apply(lambda x: x**2)
df['DrivAgeLog'] = df.DrivAge.apply(lambda x: np.log(x))
df['LicAgeLog'] = df.LicAge.apply(lambda x: np.log(x))
df['BonusMalusLog'] = df.BonusMalus.apply(lambda x: np.log(x))
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq,LicAgeSq,BonusMalusSq,DrivAgeLog,LicAgeLog,BonusMalusLog
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,0,1,0,0,2116,110224,2500,3.828641,5.805135,3.912023
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,0,1,0,0,2116,110889,2500,3.828641,5.808142,3.912023
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,1,0,0,1024,29929,4624,3.465736,5.153292,4.219508
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,1,0,0,2704,132496,2500,3.951244,5.897154,3.912023
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,1,0,3249,181476,2500,4.043051,6.054439,3.912023


In [41]:
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis = 1, inplace = True)


In [42]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['ClaimInd', 'ClaimAmount'], axis = 1), df.ClaimInd, test_size = 0.3, random_state = 1)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state = 1)

In [43]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 mins 03 secs
H2O_cluster_timezone:,Asia/Krasnoyarsk
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 27 days
H2O_cluster_name:,H2O_from_python_vlad_65zfg0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.538 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [44]:
# Преобразование в H2O-Frame

h2o_train = h2o.H2OFrame(pd.concat([x_train, y_train], axis = 1))
h2o_valid = h2o.H2OFrame(pd.concat([x_valid, y_valid], axis = 1))
h2o_test = h2o.H2OFrame(pd.concat([x_test, y_test], axis = 1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [45]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

h2o_train['ClaimInd'] = h2o_train['ClaimInd'].asfactor()
h2o_valid['ClaimInd'] = h2o_valid['ClaimInd'].asfactor()
h2o_test['ClaimInd'] = h2o_test['ClaimInd'].asfactor()

In [46]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm = H2OGeneralizedLinearEstimator(family = "binomial", link = "logit", nfolds = 5)
glm.train(y = "ClaimInd", x = h2o_train.names[:-1], training_frame = h2o_train, validation_frame = h2o_valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [47]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 1.029E-4 )",26,20,3,py_2_sid_95c0


In [48]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.71418065,0.03247624,0.72154737,0.7513719,0.73690623,0.6813659,0.6797117
1,auc,0.68431324,0.0069658463,0.6805083,0.6951528,0.6777996,0.68092585,0.68717957
2,aucpr,0.17020948,0.0058688554,0.1755873,0.1774161,0.16480832,0.16760409,0.1656316
3,err,0.28581938,0.03247624,0.27845263,0.24862808,0.2630938,0.31863406,0.3202883
4,err_count,4606.2,508.37408,4470.0,3987.0,4320.0,5188.0,5066.0
5,f0point5,0.19586043,0.011313735,0.20145106,0.2130143,0.19120927,0.18604752,0.18758
6,f1,0.2544363,0.009245954,0.25944334,0.26803744,0.24554662,0.24789794,0.2512563
7,f2,0.36409357,0.013866703,0.36432162,0.36138615,0.34302723,0.37135163,0.3803813
8,lift_top_group,2.7376368,0.26609832,2.7943664,2.697166,2.9715872,2.9226468,2.3024178
9,logloss,0.29553786,0.004972236,0.30257532,0.29764864,0.29035836,0.29581282,0.29129425


In [49]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-4.132658,-2.430618
1,Exposure,2.105119,0.608827
2,LicAge,0.000769,0.122768
3,Gender,0.000000,0.000000
4,MariStat,-0.107308,-0.038657
5,DrivAge,0.000000,0.000000
6,HasKmLimit,-0.344011,-0.107431
7,BonusMalus,0.000000,0.000000
8,OutUseNb,0.093155,0.064809
9,RiskArea,0.006938,0.015372


In [50]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm.coef_norm()
for x in range(len(glm.cross_validation_models())):
    pmodels[x] = glm.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
BonusMalus,0.00000,0.00000,0.02037,0.00000,0.00317,0.02658
BonusMalusLog,0.09951,0.11439,0.06613,0.15388,0.07437,0.05523
BonusMalusSq,0.00000,-0.02545,0.00000,-0.03802,0.02173,0.01426
DrivAge,0.00000,0.00000,0.00000,0.00000,0.00000,0.00541
DrivAgeLog,0.04642,0.00000,0.04137,0.03255,0.09013,0.01683
DrivAgeSq,-0.05961,-0.03132,-0.02739,-0.05848,-0.11243,-0.02036
Exposure,0.60883,0.61160,0.60095,0.61303,0.61063,0.60700
Gender,0.00000,0.00000,0.00280,-0.00882,0.00311,0.00315
HasKmLimit,-0.10743,-0.11808,-0.10737,-0.10736,-0.10347,-0.10127
Intercept,-2.43062,-2.44175,-2.43145,-2.42417,-2.43191,-2.42423


In [51]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

train_pred = glm.predict(h2o_train).as_data_frame()
valid_pred = glm.predict(h2o_valid).as_data_frame()
test_pred = glm.predict(h2o_test).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [54]:

# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

print(f"""Train Accuracy: {np.round(accuracy_score(y_train, train_pred["predict"].values), 4)}\n
          Valid Accuracy: {np.round(accuracy_score(y_valid, valid_pred["predict"].values), 4)}\n
          Test Accuracy: {np.round(accuracy_score(y_test, test_pred["predict"].values), 4)}\n""")


print(f"""Train F1: {np.round(f1_score(y_train, train_pred["predict"].values), 4)}\n
          Valid F1: {np.round(f1_score(y_valid, valid_pred["predict"].values), 4)}\n
          Test F1: {np.round(f1_score(y_test, test_pred["predict"].values), 4)}\n""")

print(f"""Train Precision: {np.round(precision_score(y_train, train_pred["predict"].values), 4)}\n
          Valid Precision: {np.round(precision_score(y_valid, valid_pred["predict"].values), 4)}\n
          Test Precision: {np.round(precision_score(y_test, test_pred["predict"].values), 4)}\n""")

print(f"""Train Recall: {np.round(recall_score(y_train, train_pred["predict"].values), 4)}\n
          Valid Recall: {np.round(recall_score(y_valid, valid_pred["predict"].values), 4)}\n
          Test Recall: {np.round(recall_score(y_test, test_pred["predict"].values), 4)}\n""")

print(f"""Train ROC AUC: {np.round(roc_auc_score(y_train, train_pred["p1"].values), 4)}\n
          Valid ROC AUC: {np.round(roc_auc_score(y_valid, valid_pred["p1"].values), 4)}\n
          Test ROC AUC: {np.round(roc_auc_score(y_test, test_pred["p1"].values), 4)}\n""")

print(f"""Train Log Loss: {np.round(log_loss(y_train, train_pred["p1"].values), 4)}\n
          Valid Log Loss: {np.round(log_loss(y_valid, valid_pred["p1"].values), 4)}\n
          Test Log Loss: {np.round(log_loss(y_test, test_pred["p1"].values), 4)}\n""")

Train Accuracy: 0.7166

          Valid Accuracy: 0.7179

          Test Accuracy: 0.7164

Train F1: 0.2507

          Valid F1: 0.2509

          Test F1: 0.256

Train Precision: 0.1672

          Valid Precision: 0.1667

          Test Precision: 0.17

Train Recall: 0.5006

          Valid Recall: 0.5071

          Test Recall: 0.5188

Train ROC AUC: 0.6858

          Valid ROC AUC: 0.6842

          Test ROC AUC: 0.6925

Train Log Loss: 0.2952

          Valid Log Loss: 0.2922

          Test Log Loss: 0.2924



_Судя по результатам, можно было бы почистить данные получше (в основном обратить внимание на поле Precision), попробовать изменить гипперпараметры модели, попробовать дугую модель и т.д._

